### MAP 점수 비교

In [ ]:
import json

# 두 파일에서 topk 항목을 불러옴
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Average Precision (AP) 계산
def calculate_ap(true_topk, predicted_topk):
    score = 0.0
    hits = 0
    for i, pred in enumerate(predicted_topk):
        if pred in true_topk:
            hits += 1
            score += hits / (i + 1)
    if hits == 0:
        return 0.0
    return score / len(true_topk)

def calculate_map(ground_truth_file, prediction_file, max_k=3):
    ground_truth_data = load_jsonl(ground_truth_file)
    prediction_data = load_jsonl(prediction_file)

    # 각 k별로 MAP을 저장할 딕셔너리
    map_scores_by_k = {k: [] for k in range(1, max_k + 1)}

    for gt_item in ground_truth_data:
        pred_item = next((item for item in prediction_data if item['eval_id'] == gt_item['eval_id']), None)
        if pred_item:
            for k in range(1, max_k + 1):
                top_gt = gt_item['topk'][:k]
                top_pred = pred_item['topk'][:k]
                ap = calculate_ap(top_gt, top_pred)  # 상위 k개의 항목을 사용하여 AP 계산
                map_scores_by_k[k].append(ap)

    # 각 k별로 MAP 계산
    map_scores = []
    for k in range(1, max_k + 1):
        map_score = sum(map_scores_by_k[k]) / len(map_scores_by_k[k]) if map_scores_by_k[k] else 0.0
        map_scores.append(f"top-{k}: {map_score:.4f}")

    # 한 번에 모든 top-k 결과를 출력
    result = " | ".join(map_scores)
    print(f"{prediction_file} " + result)

    return map_scores_by_k

ground_truth_file = './output/0.8614.csv'
print(f"ground_truth_file : {ground_truth_file}")
map_score = calculate_map(ground_truth_file, './output/1015-2242_topk3.csv')

### eval.jsonl에 다른 파일의 standalone_query 붙이기

In [1]:
import json

# 파일 경로 설정
a_file = './data/eval.jsonl'  # eval_id 필드가 있는 첫 번째 파일
b_file = './output/0.8409.csv'  # query 필드가 있는 두 번째 파일
output_file = './data/eval_8409.jsonl'  # 결과를 저장할 파일

# b.jsonl에서 eval_id와 standalone_query의 매핑을 생성
b_mapping = {}
with open(b_file, 'r', encoding='utf-8') as b_infile:
    for line in b_infile:
        b_doc = json.loads(line)
        # eval_id를 키로, standalone_query를 값으로 추가
        b_mapping[b_doc['eval_id']] = b_doc['standalone_query']

# a.jsonl 파일에서 eval_id에 따른 standalone_query 추가
with open(a_file, 'r', encoding='utf-8') as a_infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in a_infile:
        a_doc = json.loads(line)
        eval_id = a_doc['eval_id']
        
        # b.jsonl에서 해당 eval_id에 대한 standalone_query 가져오기
        standalone_query = b_mapping.get(eval_id, None)  # 존재하지 않을 경우 None
        
        # 새로운 문서 생성
        new_doc = {
            'eval_id': eval_id,
            'standalone_query': standalone_query,
            'msg': a_doc['msg']  # 기존 msg 필드 추가
        }
        
        # JSONL 형식으로 저장
        outfile.write(json.dumps(new_doc, ensure_ascii=False) + '\n')